# **Introduction**

This notebook documents all the measurements we created and the meanings and interpretations behind them. We use the normal times as examples.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
# pd.options.display.float_format = '{:.2f}'.format

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
link1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi.csv'
link2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_svi.csv'
normal = pd.read_csv(link1)
covid = pd.read_csv(link2)

In [6]:
normal.shape, covid.shape
# correct

((66397, 105), (5386, 104))

In [7]:
covid.columns[-13:]

Index(['average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'unemployed_impacted_nta_per_1kpop',
       'elderly_impacted_nta_per_1kpop', 'bh_impacted_nta_per_1kpop',
       'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_3year_eviction_count_per_unit(building)'],
      dtype='object')

In [8]:
# evictions per nta normalized by population
55.4/1523*1000, \
# evictions per unit (in an nta) normalized by population (*1000, *1000) double normalized for standardization
55.4/1523/39480.0*1000*1000
# correct

0.9213671358653966

In [22]:
normal[['primary_key', 'nta', 'e_totpop','borough', 'unit_per_nta', 'evictions_per_nta_1kpop(nta)', 'average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)', 'average_5year_eviction_count_per_unit(building)',
        'average_year_eviction_count_per_nta(nta)', 'average_year_eviction_per_building_nta(nta)', 'evictions_per_1kunit_nta_1kpop(nta)']].sort_values(by='evictions_per_nta_1kpop(nta)', ascending=False).head(10)

,primary_key,nta,e_totpop,borough,unit_per_nta,evictions_per_nta_1kpop(nta),average_year_eviction_count(building),average_year_eviction_count_per_unit(building),average_5year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),average_year_eviction_per_building_nta(nta),evictions_per_1kunit_nta_1kpop(nta)
14692,309061/22_210917,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,0.014286,55.4,2.946809,0.921367
1756,084970/16_202073,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,0.014286,55.4,2.946809,0.921367
39043,64622/19_493494,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,0.014286,55.4,2.946809,0.921367
8027,29668/17_335287,Crotona Park East,4292.0,BRONX,2280.0,26.281454,0.2,0.016667,0.083333,112.8,5.936842,11.526953
17201,313573/23_132495,East Concourse-Concourse Village,11280.0,BRONX,6496.0,25.496454,0.4,0.025000,0.125000,287.6,7.083744,3.924947
23092,32834/16_55430,East Concourse-Concourse Village,11280.0,BRONX,6496.0,25.496454,0.4,0.025000,0.125000,287.6,7.083744,3.924947
8317,300157/20_34280,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,147004.0,17.055074,5.0,0.019455,0.097276,326.4,5.706294,0.116018
8449,300282/21_33010,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,3.4,0.004410,0.022049,326.4,5.706294,0.038673
6753,250762/18_82775,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,3.4,0.004410,0.022049,326.4,5.706294,0.038673
6722,250616/19_95114,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,4.8,0.006226,0.031128,326.4,5.706294,0.038673


In [54]:
# evictions per nta normalized by population
11.333/1523*1000, \
# evictions per unit (in an nta) normalized by population (*1000, *1000) double normalized for standardization
11.333333/6720.0/1523.0*1000*1000, 11.333/1523*1000,
# correct, 7.44 evictions per nta 1k population

(1.1073590852953132, 7.4412344057780695)

In [24]:
covid[['primary_key', 'nta', 'e_totpop','borough', 'unit_per_nta', 'evictions_per_nta_1kpop(nta)', 'average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)', 'average_3year_eviction_count_per_unit(building)',
        'average_year_eviction_count_per_nta(nta)', 'average_year_eviction_per_building_nta(nta)', 'evictions_per_1kunit_nta_1kpop(nta)']].sort_values(by='evictions_per_nta_1kpop(nta)', ascending=False).head(10)

,primary_key,nta,e_totpop,borough,unit_per_nta,evictions_per_nta_1kpop(nta),average_year_eviction_count(building),average_year_eviction_count_per_unit(building),average_3year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),average_year_eviction_per_building_nta(nta),evictions_per_1kunit_nta_1kpop(nta)
3110,61146/19_208986,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,6720.0,7.441453,0.2,0.000952,0.002857,11.333333,3.541667,1.107359
915,301942/20_21510,Clinton,8659.0,MANHATTAN,800.0,2.386727,0.4,0.025000,0.075000,20.666667,4.133333,2.983408
982,302327/21_21550,Clinton,8659.0,MANHATTAN,800.0,2.386727,0.4,0.025000,0.075000,20.666667,4.133333,2.983408
1549,307200/21_21598,Clinton,8659.0,MANHATTAN,2200.0,2.386727,0.2,0.004545,0.013636,20.666667,4.133333,1.084876
3760,72361/19_209424,Battery Park City-Lower Manhattan,3736.0,MANHATTAN,4764.0,1.962884,0.4,0.001008,0.003023,7.333333,6.111111,0.412024
803,301267/22_111882,Battery Park City-Lower Manhattan,3736.0,MANHATTAN,4764.0,1.962884,0.4,0.001008,0.003023,7.333333,6.111111,0.412024
763,300980/21_108265,Battery Park City-Lower Manhattan,3736.0,MANHATTAN,2844.0,1.962884,0.2,0.000844,0.002532,7.333333,6.111111,0.690184
1067,302941/22_210139,Hunters Point-Sunnyside-West Maspeth,7280.0,QUEENS,30168.0,1.923077,0.6,0.000716,0.002148,14.000000,3.888889,0.063746
3407,67216/19_209116,Hunters Point-Sunnyside-West Maspeth,7280.0,QUEENS,30168.0,1.923077,0.6,0.000716,0.002148,14.000000,3.888889,0.063746
3522,68961/19_209043,Hunters Point-Sunnyside-West Maspeth,7280.0,QUEENS,18792.0,1.923077,0.2,0.000383,0.001149,14.000000,3.888889,0.102335


# **1. average_year_eviction_count(building)**

This measurement measures the average eviction per building (aggregated by all the buildings affected by evictions)

- group by bin to get total evictions per building.
- divide by the number of years (5 for noaml, 3 for covid)
- for example, if the result is 6, it means on average, this building has 6 evictions per year during normal years.

$$
\mathrm{average\ year\ eviction\ count}(\mathrm{building}) = \frac{\mathrm{Total\ Evictions}_{\mathrm{building}}}{\mathrm{Number\ of\ Years}_{\mathrm{observed}}}
$$


In [25]:
normal[['bin', 'nta', 'average_year_eviction_count(building)']]

,bin,nta,average_year_eviction_count(building)
0,3083989,East New York,0.8
1,3143881,Sunset Park East,0.6
2,3143435,Sunset Park West,0.6
3,2015444,Van Cortlandt Village,0.8
4,2003900,Mott Haven-Port Morris,1.6
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,3.8
66393,5101137,Westerleigh,0.4
66394,5101137,Westerleigh,0.4
66395,5108656,West New Brighton-New Brighton-St. George,0.2


In [26]:
covid[['bin', 'nta', 'average_year_eviction_count(building)']]

,bin,nta,average_year_eviction_count(building)
0,2113173,Kingsbridge Heights,0.2
1,4074666,Forest Hills,0.2
2,3117969,Erasmus,0.6
3,3029673,Crown Heights South,0.8
4,4079390,Kew Gardens,0.4
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.6
5382,5108502,West New Brighton-New Brighton-St. George,0.6
5383,5063024,Oakwood-Oakwood Beach,0.2
5384,5108650,West New Brighton-New Brighton-St. George,0.2


# **2. average_year_eviction_count_per_unit(building)**

This measurement measures the average eviction per building per unit(aggregated by all the buildings affected by evictions and then divide by the units in each building)

- an example would be: if the result is 0.2, it means on average per year, each unit in this building has 0.2 evictions. Or every 5 years, this building has one eviction.


$$
\text{average year eviction count per unit}(\text{building}) = \frac{\text{Average Year Eviction Counts}(\text{building})}{\text{Units in Building}}
$$


In [27]:
normal[['bin', 'nta', 'average_year_eviction_count_per_unit(building)']]

,bin,nta,average_year_eviction_count_per_unit(building)
0,3083989,East New York,0.266667
1,3143881,Sunset Park East,0.300000
2,3143435,Sunset Park West,0.150000
3,2015444,Van Cortlandt Village,0.002273
4,2003900,Mott Haven-Port Morris,0.024615
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.008370
66393,5101137,Westerleigh,0.200000
66394,5101137,Westerleigh,0.200000
66395,5108656,West New Brighton-New Brighton-St. George,0.000833


In [28]:
covid[['bin', 'nta', 'average_year_eviction_count(building)']]

,bin,nta,average_year_eviction_count(building)
0,2113173,Kingsbridge Heights,0.2
1,4074666,Forest Hills,0.2
2,3117969,Erasmus,0.6
3,3029673,Crown Heights South,0.8
4,4079390,Kew Gardens,0.4
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.6
5382,5108502,West New Brighton-New Brighton-St. George,0.6
5383,5063024,Oakwood-Oakwood Beach,0.2
5384,5108650,West New Brighton-New Brighton-St. George,0.2


# **3. average_5year_eviction_count_per_unit(building)**
# **average_3year_eviction_count_per_unit(building) -- for covid**

## **Interpretations**
- Measures eviction intensity per unit for every 5 years (during normal times) / 3 years (during covid)
- Answers the question:"In a particular building, how many evictions does one unit has on average every 5 years?"
- For example, 1.33 outside of covid means, on average, 1.33 evictions occur at this unit in this building every 5 years.


$$
\text{average year eviction per unit every 5 years} = \left( \frac{\text{Average Eviction Counts per Building per year}}{\text{Unit Counts in that Building}} \right) \times 5 (\text{Years})
$$


In [29]:
normal[['bin', 'average_year_eviction_count(building)', 'unitsres', 'average_year_eviction_count_per_unit(building)', 'average_5year_eviction_count_per_unit(building)']].head()

,bin,average_year_eviction_count(building),unitsres,average_year_eviction_count_per_unit(building),average_5year_eviction_count_per_unit(building)
0,3083989,0.8,3.0,0.266667,1.333333
1,3143881,0.6,2.0,0.300000,1.500000
2,3143435,0.6,4.0,0.150000,0.750000
3,2015444,0.8,352.0,0.002273,0.011364
4,2003900,1.6,65.0,0.024615,0.123077


In [30]:
covid[['bin', 'average_year_eviction_count(building)', 'unitsres', 'average_year_eviction_count_per_unit(building)', 'average_3year_eviction_count_per_unit(building)']].head()

,bin,average_year_eviction_count(building),unitsres,average_year_eviction_count_per_unit(building),average_3year_eviction_count_per_unit(building)
0,2113173,0.2,3.0,0.066667,0.200000
1,4074666,0.2,181.0,0.001105,0.003315
2,3117969,0.6,2.0,0.300000,0.900000
3,3029673,0.8,48.0,0.016667,0.050000
4,4079390,0.4,79.0,0.005063,0.015190


# **4. average_year_eviction_count_per_nta(nta)**

This measurement measures the average eviction per neighborhood(aggregated by all the evictions happened within a neighborhood and divided by the number of years observed.)

- an example would be: if the result is 120, it means on average, this neighborhood has 120 evictions per year.

$$
\text{average yearly evictions}(\text{neighborhood}) = \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Number of Years Observed}}
$$

In [31]:
normal[['bin', 'nta', 'average_year_eviction_count_per_nta(nta)']]

,bin,nta,average_year_eviction_count_per_nta(nta)
0,3083989,East New York,266.0
1,3143881,Sunset Park East,38.0
2,3143435,Sunset Park West,39.6
3,2015444,Van Cortlandt Village,172.8
4,2003900,Mott Haven-Port Morris,158.8
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,89.6
66393,5101137,Westerleigh,6.8
66394,5101137,Westerleigh,6.8
66395,5108656,West New Brighton-New Brighton-St. George,89.6


In [32]:
covid[['bin', 'nta', 'average_year_eviction_count_per_nta(nta)']]

,bin,nta,average_year_eviction_count_per_nta(nta)
0,2113173,Kingsbridge Heights,18.666667
1,4074666,Forest Hills,6.000000
2,3117969,Erasmus,23.000000
3,3029673,Crown Heights South,23.000000
4,4079390,Kew Gardens,6.000000
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,20.333333
5382,5108502,West New Brighton-New Brighton-St. George,20.333333
5383,5063024,Oakwood-Oakwood Beach,5.000000
5384,5108650,West New Brighton-New Brighton-St. George,20.333333


# **5. evictions_per_nta_1kpop(nta)**

This measurement measures the average eviction per neighborhood(aggregated by all the evictions happened within a neighborhood and divided by the number of years observed.) and normalized by every 1000 population.

- an example would be: if the result is 2.74, it means on average, every 1000 people living in this neighborhood, 2.74 of them would get eviction per year.


$$
\text{eviction rate per 1k population}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Population}} \right) \times 1000
$$

In [33]:
normal[['bin', 'nta', 'evictions_per_nta_1kpop(nta)']]

,bin,nta,evictions_per_nta_1kpop(nta)
0,3083989,East New York,2.747905
1,3143881,Sunset Park East,0.408567
2,3143435,Sunset Park West,0.425770
3,2015444,Van Cortlandt Village,2.122928
4,2003900,Mott Haven-Port Morris,3.578188
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,2.221616
66393,5101137,Westerleigh,0.366241
66394,5101137,Westerleigh,0.366241
66395,5108656,West New Brighton-New Brighton-St. George,3.414764


In [34]:
covid[['bin', 'nta', 'evictions_per_nta_1kpop(nta)']]

,bin,nta,evictions_per_nta_1kpop(nta)
0,2113173,Kingsbridge Heights,0.229329
1,4074666,Forest Hills,0.079775
2,3117969,Erasmus,0.227603
3,3029673,Crown Heights South,0.393324
4,4079390,Kew Gardens,0.295348
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.504161
5382,5108502,West New Brighton-New Brighton-St. George,0.504161
5383,5063024,Oakwood-Oakwood Beach,0.088917
5384,5108650,West New Brighton-New Brighton-St. George,0.774928


# **6. evictions_per_1kunit_nta_1kpop(nta)**

This measurement measures the average eviction per unit in each neighborhood(aggregated by all the evictions happened within a neighborhood and divided by all the units) and normalized by every 1000 population and every 1000 units.

- an example would be: if the result is 0.0217, it means every 1000 units in that neighborhood, we observe on average 0.0217 evictions  

$$
\text{eviction rate per 1k population}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Population}} \right) \times 1000
$$

In [35]:
normal[['bin', 'nta', 'evictions_per_1kunit_nta_1kpop(nta)']]

,bin,nta,evictions_per_1kunit_nta_1kpop(nta)
0,3083989,East New York,1.263405
1,3143881,Sunset Park East,1.502085
2,3143435,Sunset Park West,0.724098
3,2015444,Van Cortlandt Village,0.024717
4,2003900,Mott Haven-Port Morris,0.170431
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.018259
66393,5101137,Westerleigh,6.104020
66394,5101137,Westerleigh,6.104020
66395,5108656,West New Brighton-New Brighton-St. George,0.053090


In [36]:
covid[['bin', 'nta', 'evictions_per_1kunit_nta_1kpop(nta)']]

,bin,nta,evictions_per_1kunit_nta_1kpop(nta)
0,2113173,Kingsbridge Heights,2.011655
1,4074666,Forest Hills,0.025926
2,3117969,Erasmus,2.473949
3,3029673,Crown Heights South,0.190564
4,4079390,Kew Gardens,0.311549
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.027085
5382,5108502,West New Brighton-New Brighton-St. George,0.027085
5383,5063024,Oakwood-Oakwood Beach,0.617482
5384,5108650,West New Brighton-New Brighton-St. George,0.078753


# **7. average_year_eviction_per_building_nta(nta)**

This measurement measures the average evcitions per 10 building within particular neighborhood.

- an example would be: if the result for an nta is 3.668966, it means, for this particular neighborhood, every 10 buildings, they have 3.668966 evictions on average per year.

$$
\text{evictions per year per 10 buildings}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Total Buildings}(\text{neighborhood})} \right) \times 10
$$

In [37]:
normal[['bin', 'nta', 'average_year_eviction_per_building_nta(nta)']]

,bin,nta,average_year_eviction_per_building_nta(nta)
0,3083989,East New York,3.668966
1,3143881,Sunset Park East,2.794118
2,3143435,Sunset Park West,2.693878
3,2015444,Van Cortlandt Village,7.081967
4,2003900,Mott Haven-Port Morris,4.916409
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,3.343284
66393,5101137,Westerleigh,2.266667
66394,5101137,Westerleigh,2.266667
66395,5108656,West New Brighton-New Brighton-St. George,3.343284


In [38]:
covid[['bin', 'nta', 'average_year_eviction_per_building_nta(nta)']]

,bin,nta,average_year_eviction_per_building_nta(nta)
0,2113173,Kingsbridge Heights,4.912281
1,4074666,Forest Hills,3.529412
2,3117969,Erasmus,5.000000
3,3029673,Crown Heights South,5.348837
4,4079390,Kew Gardens,5.000000
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,4.959350
5382,5108502,West New Brighton-New Brighton-St. George,4.959350
5383,5063024,Oakwood-Oakwood Beach,5.555556
5384,5108650,West New Brighton-New Brighton-St. George,4.959350


# **8. children_impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many children (under 17) were affected by evictions per year.

- for example, if an nta has 0.618279, this means, for every 1000 people living this neighborhood, about 0.618279 child was affected by evictions.

$$
\text{children impacted per 1k pop}(\text{neighborhood}) = \left ( \text{Evictions per 1k}(\text{neighborhood}) \times \frac {\text{Children Percentage}(\text{neighborhood})} {100} \right)
$$

In [39]:
normal[['bin', 'nta', 'children_impacted_nta_per_1kpop']]

,bin,nta,children_impacted_nta_per_1kpop
0,3083989,East New York,0.618279
1,3143881,Sunset Park East,0.103776
2,3143435,Sunset Park West,0.108146
3,2015444,Van Cortlandt Village,0.560453
4,2003900,Mott Haven-Port Morris,1.005471
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.459875
66393,5101137,Westerleigh,0.086799
66394,5101137,Westerleigh,0.086799
66395,5108656,West New Brighton-New Brighton-St. George,0.877594


In [40]:
covid[['bin', 'nta', 'children_impacted_nta_per_1kpop']]

,bin,nta,children_impacted_nta_per_1kpop
0,2113173,Kingsbridge Heights,0.060543
1,4074666,Forest Hills,0.014359
2,3117969,Erasmus,0.042562
3,3029673,Crown Heights South,0.065685
4,4079390,Kew Gardens,0.053163
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.104361
5382,5108502,West New Brighton-New Brighton-St. George,0.104361
5383,5063024,Oakwood-Oakwood Beach,0.018939
5384,5108650,West New Brighton-New Brighton-St. George,0.199156


# **9. elderly_impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many elderly (over 65) were affected by evictions per year.

- for example, if an nta has 0.379211, this means, for every 1000 people living this neighborhood, about 0.379211 elderly was affected by evictions.

$$
\text{elderly impacted per 1k pop}(\text{neighborhood}) = \left ( \text{Evictions per 1k}(\text{neighborhood}) \times \frac {\text{Elderly Percentage}(\text{neighborhood})} {100} \right)
$$

In [41]:
normal[['bin', 'nta', 'elderly_impacted_nta_per_1kpop']]

,bin,nta,elderly_impacted_nta_per_1kpop
0,3083989,East New York,0.379211
1,3143881,Sunset Park East,0.053522
2,3143435,Sunset Park West,0.055776
3,2015444,Van Cortlandt Village,0.237768
4,2003900,Mott Haven-Port Morris,0.361397
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.346572
66393,5101137,Westerleigh,0.037357
66394,5101137,Westerleigh,0.037357
66395,5108656,West New Brighton-New Brighton-St. George,0.430260


In [42]:
covid[['bin', 'nta', 'elderly_impacted_nta_per_1kpop']]

,bin,nta,elderly_impacted_nta_per_1kpop
0,2113173,Kingsbridge Heights,0.025685
1,4074666,Forest Hills,0.016274
2,3117969,Erasmus,0.029816
3,3029673,Crown Heights South,0.060179
4,4079390,Kew Gardens,0.050209
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.078649
5382,5108502,West New Brighton-New Brighton-St. George,0.078649
5383,5063024,Oakwood-Oakwood Beach,0.016716
5384,5108650,West New Brighton-New Brighton-St. George,0.097641


# **10. black + hispanic _impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many black and hispanic population were affected by evictions per year.

- for example, if an nta has 2.437392, this means, for every 1000 people living this neighborhood, about 2.437392 black and hispanic people were affected by evictions.

$$
\text{black and hispanic impacted per 1k pop}(\text{neighborhood}) = \left ( \text{Evictions per 1k}(\text{neighborhood}) \times \frac {\text{Black and Hispanic Population Percentage}(\text{neighborhood})} {100} \right)
$$

In [43]:
normal[['bin', 'nta', 'bh_impacted_nta_per_1kpop']]

,bin,nta,bh_impacted_nta_per_1kpop
0,3083989,East New York,2.437392
1,3143881,Sunset Park East,0.174050
2,3143435,Sunset Park West,0.181378
3,2015444,Van Cortlandt Village,1.987061
4,2003900,Mott Haven-Port Morris,3.406435
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,1.026387
66393,5101137,Westerleigh,0.232929
66394,5101137,Westerleigh,0.232929
66395,5108656,West New Brighton-New Brighton-St. George,1.673234


In [44]:
covid[['bin', 'nta', 'bh_impacted_nta_per_1kpop']]

,bin,nta,bh_impacted_nta_per_1kpop
0,2113173,Kingsbridge Heights,0.214652
1,4074666,Forest Hills,0.015237
2,3117969,Erasmus,0.177758
3,3029673,Crown Heights South,0.253694
4,4079390,Kew Gardens,0.087423
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.232923
5382,5108502,West New Brighton-New Brighton-St. George,0.232923
5383,5063024,Oakwood-Oakwood Beach,0.017072
5384,5108650,West New Brighton-New Brighton-St. George,0.379715


# **11. unemployed _impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many unemployed were affected by evictions per year.

- for example, if an nta has 0.305018, this means, for every 1000 people living this neighborhood, about 0.305018 unemployed person was affected by evictions.

$$
\text{unemployed impacted per 1k pop}(\text{neighborhood}) = \left ( \text{Evictions per 1k}(\text{neighborhood}) \times \frac {\text{Unemployed Population Percentage}(\text{neighborhood})} {100} \right)
$$

In [45]:
normal[['bin', 'nta', 'unemployed_impacted_nta_per_1kpop']]

,bin,nta,unemployed_impacted_nta_per_1kpop
0,3083989,East New York,0.305018
1,3143881,Sunset Park East,0.030643
2,3143435,Sunset Park West,0.031933
3,2015444,Van Cortlandt Village,0.246260
4,2003900,Mott Haven-Port Morris,0.447274
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.157735
66393,5101137,Westerleigh,0.028567
66394,5101137,Westerleigh,0.028567
66395,5108656,West New Brighton-New Brighton-St. George,0.143420


In [46]:
covid[['bin', 'nta', 'unemployed_impacted_nta_per_1kpop']]

,bin,nta,unemployed_impacted_nta_per_1kpop
0,2113173,Kingsbridge Heights,0.026602
1,4074666,Forest Hills,0.003829
2,3117969,Erasmus,0.013429
3,3029673,Crown Heights South,0.025959
4,4079390,Kew Gardens,0.016540
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.035795
5382,5108502,West New Brighton-New Brighton-St. George,0.035795
5383,5063024,Oakwood-Oakwood Beach,0.005957
5384,5108650,West New Brighton-New Brighton-St. George,0.032547


# **12. burdened_househould_impacted_nta_per_1kpop**

Any household who spend more than 30% of income on housing costs are considered housing-burdened.

This measurement measures that for each neighborhood, for every 1000 population, how many households that already spent more than 30% of income on housing costs were affected by evictions per year.

- for example, if an nta has 1.228314, this means, for every 1000 people living this neighborhood, about 1.228314 hburd households were affected by evictions.

$$
\text{hburd impacted per 1k pop}(\text{neighborhood}) = \left ( \text{Evictions per 1k}(\text{neighborhood}) \times \frac {\text{hburd Population Percentage}(\text{neighborhood})} {100} \right)
$$

In [47]:
normal[['bin', 'nta', 'hburd_impacted_nta_per_1kpop']]

,bin,nta,hburd_impacted_nta_per_1kpop
0,3083989,East New York,1.228314
1,3143881,Sunset Park East,0.178135
2,3143435,Sunset Park West,0.185636
3,2015444,Van Cortlandt Village,1.203700
4,2003900,Mott Haven-Port Morris,1.857080
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.715360
66393,5101137,Westerleigh,0.129283
66394,5101137,Westerleigh,0.129283
66395,5108656,West New Brighton-New Brighton-St. George,0.956134


In [48]:
covid[['bin', 'nta', 'hburd_impacted_nta_per_1kpop']]

,bin,nta,hburd_impacted_nta_per_1kpop
0,2113173,Kingsbridge Heights,0.130029
1,4074666,Forest Hills,0.020263
2,3117969,Erasmus,0.089221
3,3029673,Crown Heights South,0.146710
4,4079390,Kew Gardens,0.095397
...,...,...,...
5381,5108502,West New Brighton-New Brighton-St. George,0.162340
5382,5108502,West New Brighton-New Brighton-St. George,0.162340
5383,5063024,Oakwood-Oakwood Beach,0.024897
5384,5108650,West New Brighton-New Brighton-St. George,0.216980
